In [35]:
import tensorflow as tf
!pip install tflite-model-maker


!pip install six
!pip install google-api-python-client
!pip install google-cloud-bigquery
!pip install kaggle
# !pip install numpy>=1.15.4
# !pip install pandas>=0.22.0
!pip install psutil
!pip install py-cpuinfo
!pip install scipy
!pip install tensorflow-hub
!pip install tensorflow-model-optimization
!pip install tensorflow-datasets
!pip install tensorflow-addons
!pip install dataclasses
!pip install gin-config
!pip install tf_slim
!pip install sentencepiece
!pip install Cython
!pip install matplotlib
!pip install opencv-python-headless
!pip install pyyaml
!pip install Pillow
!pip install -e 'git+https://github.com/cocodataset/cocoapi#egg=pycocotools&subdirectory=PythonAPI'


!pip install -q -U "tensorflow-text==2.10.*"
!pip install -q tf-models-official==2.10
!pip install --upgrade tensorflow==2.10

!pip install cupy-cuda115
!pip install --upgrade numba 

# !pip install numpy==1.22
!pip install gensim



  Using cached tflite_model_maker-0.4.2-py3-none-any.whl (577 kB)
  Using cached fire-0.5.0-py2.py3-none-any.whl
  Using cached tf_models_official-2.3.0-py2.py3-none-any.whl (840 kB)
  Using cached tflite_support-0.4.3-cp38-cp38-manylinux2014_x86_64.whl (60.8 MB)
  Using cached scann-1.2.6-cp38-cp38-manylinux2014_x86_64.whl (10.9 MB)
  Using cached numba-0.53.0-cp38-cp38-manylinux2014_x86_64.whl (3.4 MB)
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Using cached matplotlib-3.4.3-cp38-cp38-manylinux1_x86_64.whl (10.3 MB)
  Using cached tensorflowjs-3.18.0-py3-none-any.whl (77 kB)
  Using cached neural_structured_learning-1.4.0-py2.py3-none-any.whl (128 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
  Using cached llvmlite-0.36.0-cp38-cp38-manylinux2010_x86_64.whl (25.3 MB)
  Using cached tensorflow-2.8.4-cp38-cp38-manylinux2010_x86_64.whl

In [36]:
# !pip install numpy==1.23
# !pip install --upgrade numba
!pip install transformers


In [3]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

2023-04-26 11:11:11.399488: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [1]:


import pandas as pd
import torch
# from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import tensorflow as tf


import pandas as pd
import numpy as np
import matplotlib
# import seaborn as sns

# from gensim.models.word2vec import Word2Vec
import matplotlib.pyplot as plt
import pickle
import warnings

# !pip install nltk==3.5
# import nltk
# from nltk.stem import PorterStemmer #for stemming
# from nltk.tokenize import word_tokenize
# import nltk.corpus

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import  LogisticRegression
# from sklearn.linear_model import SGDClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import KFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import learning_curve
# from sklearn.metrics import confusion_matrix, f1_score, classification_report
# from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import average_precision_score
# from sklearn import svm



# !pip install 

warnings.filterwarnings("ignore")

2023-04-26 11:35:53.098229: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 11:35:53.277851: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-26 11:35:54.246011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bitfusion//4.5.0-dc222ab7/x86_64-linux-gnu//lib/nvml:/opt/bitfusion//4.5.0-dc222ab7/x86_64-linux-gnu//lib/cuda:/opt/bitfusion//4.5.0-dc222ab7/x86_64-linux-gnu//lib:
2023-04-26 11:35:54.246179: W tensorflow/s

In [2]:
import tensorflow_models as tfm
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [3]:
path_to_train = "train.tsv"
path_to_test = "test.tsv"
path_to_valid = "valid.tsv"
df_train = pd.read_csv(path_to_train, delimiter='\t', dtype=object)
df_test = pd.read_csv(path_to_test, delimiter='\t', dtype=object)
df_valid = pd.read_csv(path_to_valid, delimiter='\t', dtype=object)

In [4]:
df_train.fillna("", inplace=True)
df_train.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement).
    ]


df_test.fillna("", inplace=True)
df_test.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement).
    ]

df_valid.fillna("", inplace=True)
df_valid.columns = [
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement).
    ]

In [5]:
df_train = df_train.drop(columns=["id", "subjects", "speaker", "speaker_job_title", "state_info", "party_affiliation", "count_1", "count_2", "count_3", "count_4", "count_5", "context"])
df_train = df_train.replace({'label': {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}})
df_train = df_train.dropna()
train_labels = df_train["label"].values
train_sentences = df_train["statement"].values


df_valid = df_valid.drop(columns=["id", "subjects", "speaker", "speaker_job_title", "state_info", "party_affiliation", "count_1", "count_2", "count_3", "count_4", "count_5", "context"])
df_valid = df_valid.replace({'label': {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}})
df_valid = df_valid.dropna()
valid_labels = df_train["label"].values
valid_sentences = df_train["statement"].values

df_test = df_test.drop(columns=["id", "subjects", "speaker", "speaker_job_title", "state_info", "party_affiliation", "count_1", "count_2", "count_3", "count_4", "count_5", "context"])
df_test = df_test.replace({'label': {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}})
df_test = df_test.dropna()
test_labels = df_test["label"].values
test_sentences = df_test["statement"].values


In [6]:
# # Load BERT tokenizer and tokenize data
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# # Get encodings
# train_encodings = tokenizer(train_sentences.tolist(), truncation=True, padding=True)
# val_encodings = tokenizer(valid_sentences.tolist(), truncation=True, padding=True)
# test_encodings = tokenizer(test_sentences.tolist(), truncation=True, padding=True)

In [7]:
import tensorflow as tf

In [8]:
with tf.device('/cpu:0'):
  train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels))
  valid_dataset = tf.data.Dataset.from_tensor_slices((valid_sentences, valid_labels))

2023-04-26 11:36:09.739217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 11:36:09.794301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 11:36:09.795466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 11:36:09.797776: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
# # Convert data to PyTorch tensors
# train_inputs = {key: torch.tensor(val) for key, val in train_encodings.items()}
# train_labels = torch.tensor(train_labels)
# val_inputs = {key: torch.tensor(val) for key, val in val_encodings.items()}
# val_labels = torch.tensor(val_labels)

In [10]:
# ########Convert to tf tensors

# import tensorflow as tf

# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_labels
# ))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings),
#     test_labels
# ))


# # valid_dataset = tf.data.Dataset.from_tensor_slices((
# #     dict(val_encodings),
# #     valid_labels
# # ))


# valid_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings)
# ))

In [11]:
# # Define model
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6, output_attentions=False, output_hidden_states=False)

In [12]:
# # Define optimizer and learning rate
# optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
# num_epochs = 4
# num_training_steps = num_epochs * len(train_inputs)
# num_warmup_steps = int(0.1 * num_training_steps)
# lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

In [13]:
# # Train
# model.cuda()
# model.train()
# for epoch in range(num_epochs):
#     for i in range(0, len(train_inputs), 32):
#         optimizer.zero_grad()
#         batch_inputs = {key: val[i:i+32].cuda() for key, val in train_inputs.items()}
#         batch_labels = train_labels[i:i+32].cuda()
#         outputs = model(**batch_inputs, labels=batch_labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         lr_scheduler.step()

#     # Evaluate on validation set
#     model.eval()
#     val_loss = 0
#     val_correct = 0
#     with torch.no_grad():
#         for i in range(0, len(val_inputs), 32):
#             batch_inputs = {key: val[i:i+32].cuda() for key, val in val_inputs.items()}
#             batch_labels = val_labels[i:i+32].

In [14]:
# from transformers import TFBertForSequenceClassification


# # Fine-tune the pre-trained BERT model on the training data
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
# model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
# history = model.fit(train_dataset.batch(16), epochs=5, validation_data=valid_dataset.batch(16))

# # Plot the training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']
# epochs = range(1, len(train_loss) + 1)

# plt.plot(epochs, train_loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # Evaluate the fine-tuned model on the test data
# test_predictions = model.predict(test_dataset.batch(16)).logits
# test_predictions = tf.nn.softmax(test_predictions, axis=1).numpy()
# test_predicted_labels = test_predictions.argmax(axis=1)

In [15]:
from transformers import TFBertForSequenceClassification

In [16]:
# def binary_crossentropy_with_logits(labels, logits):
#     labels = tf.cast(labels, tf.float32)
#     return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))


In [17]:
# # Load BERT tokenizer and tokenize data
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [18]:
# Load the BERT model

num_classes = 6
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=num_classes)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# !git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git

In [20]:
# !pip install https://github.com/tensorflow/models.git

In [28]:
import sys
sys.path.append('models/official/')
# sys.path.append('models/official/nlp/bert')

from official.nlp.data import classifier_data_lib
from official.nlp import optimization


from official.nlp.bert import tokenization
# sys.path.append('models/official/nlp/bert/tokenization')

# RandomForest/models/official/nlp/bert/tokenization.py

In [30]:
# sys.path.append('RandomForest/models/official/nlp/bert')
# !cd RandomForest/models/official/nlp/bert
# from official.nlp.bert import tokenization

In [31]:
import os

In [32]:
label_list = [0, 1,2,3,4,5]
max_seq_length =  128
batch_size=32


# from official.nlp.bert import tokenization
tokenizer = FullTokenizer(vocab_file, do_lower_case)

# tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
#     vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
#     lower_case=True)

In [33]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2",
                            trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [34]:
# This provides a function to convert row to input features and label

def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
  example = classifier_data_lib.InputExample(guid = None,
                                            text_a = text.numpy(), 
                                            text_b = None, 
                                            label = label.numpy())
  feature = classifier_data_lib.convert_single_example(0, example, label_list,
                                    max_seq_length, tokenizer)
  
  return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

In [35]:
def to_feature_map(text, label):
  input_ids, input_mask, segment_ids, label_id = tf.py_function(to_feature, inp=[text, label], 
                                Tout=[tf.int32, tf.int32, tf.int32, tf.int32])

  # py_func doesn't set the shape of the returned tensors.
  input_ids.set_shape([max_seq_length])
  input_mask.set_shape([max_seq_length])
  segment_ids.set_shape([max_seq_length])
  label_id.set_shape([])

  x = {
        'input_word_ids': input_ids,
        'input_mask': input_mask,
        'input_type_ids': segment_ids
    }
  return (x, label_id)


In [36]:
# train_dataset = train_dataset.shuffle(len(train_sentences)).batch(8).cache().prefetch(tf.data.AUTOTUNE)
# test_dataset = test_dataset.batch(8).cache().prefetch(tf.data.AUTOTUNE)
# valid_dataset = valid_dataset.batch(8).cache().prefetch(tf.data.AUTOTUNE)

In [37]:
with tf.device('/cpu:0'):
  # train
  train_dataset = (train_dataset.map(to_feature_map,
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          #.cache()
                          .shuffle(1000)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE))

  # valid
  valid_dataset = (valid_dataset.map(to_feature_map,
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE)) 

In [38]:

# device = tf.device('/gpu:0')

# # bert_model.compile(optimizer=optimizer, loss=binary_crossentropy_with_logits, metrics=['accuracy'])

# # Compile the model
# # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
# # bert_model.layers[-1].activation = tf.keras.activations.softmax
# # bert_model.compile(optimizer=optimizer, loss=tf.nn.sigmoid_cross_entropy_with_logits, metrics=['accuracy'])

# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=1e-08, clipnorm=1.0)
# bert_model.compile(optimizer=optimizer, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])



# # Train the model
# history = bert_model.fit(train_dataset, epochs=5, validation_data=valid_dataset)

# # Plot the training and validation loss
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']
# epochs = range(1, len(train_loss) + 1)

# plt.plot(epochs, train_loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [39]:
# Building the model
def create_model():
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                  name="input_mask")
  input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                  name="input_type_ids")

  pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

  drop = tf.keras.layers.Dropout(0.4)(pooled_output)
  output = tf.keras.layers.Dense(6, activation="softmax", name="output")(drop)

  model = tf.keras.Model(
    inputs={
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids
    },
    outputs=output)
  return model

In [42]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 128, 768)]                'input_mask[0][0]',       

In [43]:
device = tf.device('/gpu:0')

epochs=4
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=epochs,
                    verbose=1)

Epoch 1/4


2023-04-26 11:44:45.112245: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 24.00MiB (rounded to 25165824)requested by op model_1/keras_layer_1/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/transformer_encoder/StatefulPartitionedCall/transformer/layer_7/self_attention/einsum/Einsum
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-26 11:44:45.112467: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-04-26 11:44:45.112511: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 235, Chunks in use: 234. 58.8KiB allocated for chunks. 58.5KiB in use in bin. 1.7KiB client-requested in use in bin.
2023-04-26 11:44:45.112540: I tensorflow/core/common_runtime/bfc_allocator.c

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[32,12,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node transformer/layer_7/self_attention/einsum/Einsum}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_74982]

.cc:1089] InUse at 7f880a1d6600 of size 2359296 next 1206
2023-04-26 11:44:45.126227: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880a416600 of size 3072 next 1207
2023-04-26 11:44:45.126233: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880a417200 of size 2359296 next 1208
2023-04-26 11:44:45.126239: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880a657200 of size 3072 next 1209
2023-04-26 11:44:45.126245: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880a657e00 of size 2359296 next 1210
2023-04-26 11:44:45.126251: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880a897e00 of size 3072 next 1211
2023-04-26 11:44:45.126256: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880a898a00 of size 2359296 next 1212
2023-04-26 11:44:45.126262: I tensorflow/core/common_runtime/bfc_allocator.cc:1089] InUse at 7f880aad8a00 of size 3072 next 1213
2023-04-26 11:44:45.126269: I 